Long-term create a jupyter kernel for any REPL interface

https://github.com/n-riesco/nel/blob/master/lib/nel.js

https://github.com/n-riesco/jp-kernel/blob/master/lib/jp-kernel.js

https://github.com/takluyver/bash_kernel/blob/master/bash_kernel/kernel.py

https://github.com/n-riesco/ijavascript/blob/d459956f76a22f9ec89937e02645ee7555f92d2b/lib/kernel.js

https://github.com/notablemind/jupyter-nodejs/blob/1388e1957792ef167ccfc7a9cf6c64c3950ded76/lib/kernel.js

https://github.com/JuliaLang/IJulia.jl

https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js

looks like it is just passed to the command line in rust

https://github.com/google/evcxr/blob/602db3ef52bfddeb34608877bd72b9d0d112fa26/evcxr/src/module.rs


Fix ijavascript not showing filename

https://nodejs.org/api/vm.html#vm_class_vm_script

Fix the problem with ijavascript not supporting %%

Go through this list and find demo code for every language and make sure it works with our REPL, minimize dependencies

https://github.com/jupyter/jupyter/wiki/Jupyter-kernels

https://jupyter-client.readthedocs.io/en/stable/kernels.html

https://jupyter-client.readthedocs.io/en/stable/wrapperkernels.html


Add HTML, CSS/SCSS kernels with some sort of visual output specially make for the file type

Create a kernels notebook





Goal: rewrite entire nel, jupyter-nodejs, jp-babel, jp-kernel functionality in one notebook and a few lines of code

Goal: transpile the kernel in to the native language using AST translations

TODO: implement both meta kernels in python and socket based kernels (through library include in another language or through command like repl interface), translate zmq to every language

TODO: use syntax highlighter to "train" a transpiler to write socket based kernels in any language, at least 
BROWSER C# GO JAVA NODE.JS PHP PYTHON RUBY from Google

TODO: make a processing kernel https://github.com/processing/p5.js/wiki/p5.js,-node.js,-socket.io


get kernel json?


In [ ]:
var path = require('path');

function kernelJsonTemplate(kernel_json) {
    return {
        argv,
        display_name,
        language,
        metadata,
        env,
        interrupt_mode
    } = kernel_json
}

function standardJsonTemplate(kernel_json) {
    return kernelJsonTemplate(Object.assign({
//        argv: ['python', '-m', kernel_json.path, '-f', '{connection_file}']
        // TODO: replace with path
        argv: ['npm', 'run', 'import',
               '--', 'get javascript kernel', '["{connection_file}"]'],
    }, kernel_json))
}

module.exports = {
    kernelJsonTemplate,
    standardJsonTemplate
};


get meta kernel?


In [6]:

function kernelMetaTemplate(meta_kernel) {
    return {
        do_execute,
        do_complete,
        do_inspect,
        do_history,
        do_is_complete,
        do_shutdown
    } = meta_kernel
}

function javascriptMeta(meta_kernel) {
    return Object.assign({
        do_init: (kernel, config) => {
            process.on('message', (message) => {
                kernel[Object.keys(message)[0]].apply(kernel,
                                                      [kernel].concat(Object.values(message)));
            })
            process.send({status: "online"})
        },
        do_execute: (kernel, request) => {
            var result = require('vm').runInThisContext(request.code);
            process.send({mime: {
                'text/plain': result + '',
                execution_count: request.execution_count
            }})
            process.send({display: {
                'text/plain': result + '',
                execution_count: request.execution_count
            }})
            //process.send({display: {mime: result}})
        },
        do_shutdown: (kernel, request) => {},
        do_is_complete: (kernel, request) => {
            try {
                kernel.do_execute(request);
                return true;
            } catch {
                return false;
            }
        },
        do_complete: (kernel, request) => {
            // TODO: call lang-server code
            throw new Error('not implemented!');
        },
        do_history: (kernel, request) => {
            throw new Error('not implemented!');
        },
        do_inspect: (kernel, request) => {
            // TODO: call lang-server code
            throw new Error('not implemented!');
        }
    }, meta_kernel, meta_kernel.kernel_info)
}

module.exports = {
    kernelMetaTemplate,
    javascriptMeta
};


{ kernelMetaTemplate: [Function: kernelMetaTemplate],
  javascriptMeta: [Function: javascriptMeta] }

get kernel language?


In [ ]:

function languageTemplate(language_info) {
    return {
        mimetype,
        name,
        file_extension,
        version,
        pygments_lexer,
        codemirror_mode,
        nbconvert_exporter
    } = language_info
}

module.exports = languageTemplate;


 get kernel info?
 

In [ ]:

function kernelInfoTemplate(kernel_info) {
    return {
        protocol_version,
        implementation,
        implementation_version,
        banner,
        language_info,
        help_links
    } = kernel_info
}

module.exports = kernelInfoTemplate;


get kernel session?


In [3]:
var importer = require('../Core');
var crypto = require('crypto');
var uuid = require('uuid/v4');
var {setupSockets} = importer.import('bind to jupyter zmq sockets');

function kernelSessionTemplate(session) {
    return {
        init,
        message,
        respond,
        status,
        display,
        mime,
        startup,
        execute_request,
        inspect_request,
        complete_request,
        history_request,
        kernel_info_request,
        is_complete_request,
        connect_request,
        comm_info_request,
        kernel_info_request,
        shutdown_request,
        interrupt_request,
        input_request
    } = session
}

// uuids, delim, hmac, header, parent_header, metadata, content
var DELIM = '<IDS|MSG>'
function parseMessage(cb, respond) {
    const strs = [].map.call(arguments, a => a.toString())
    let i
    for (i=0; i<strs.length; i++) {
      if (strs[i] === DELIM) {
        break
      }
    }
    const uuids = [].slice.call(arguments, 2, i)
    const args = strs.slice(i + 2).map(a => JSON.parse(a))
    let [header, parent, metadata, content] = args
    var result = {};
    result[header.msg_type] = {
        content,
        respond: respond.bind(null, {uuids, header: Object.assign({}, header), parent, metadata})
    };
    return cb(result);
}

function hash(string, key) {
    const hmac = crypto.createHmac('sha256', key)
    hmac.update(string)
    const res = hmac.digest('hex')
    return res
}

function json(data) {
    return JSON.stringify(data).replace('\ufdd0', '\\ufdd0')
}

function collapseMessage(key, cb, response, content) {
    response.header.msg_id = uuid();
    response.header.msg_type = Object.keys(content)[0];
    const toHash = [
      json(response.header),
      json(response.parent),
      json(response.metadata || {}),
      ...Object.values(content).map(a => json(a))]
    const hmac = hash(toHash.join(''), key)
    const args = response.uuids.concat([DELIM, hmac]).concat(toHash);
    return cb(content, args)
}

var count = 0;
var responders = [];
function standardMessaging(session) {
    return Object.assign({
        init: (kernel, config) => {
            var session = kernel.session;
            kernel.config = config;
            console.log('spawning child process');
            var meta_kernel = Object.keys(kernel.meta_kernel)
                .map(k => `'${k}': (${kernel.meta_kernel[k].toString()})`)
                .join(',');
            session.child = require("child_process")
                // TODO: replace this with configurable node path
                .spawn(process.argv[0], ['--eval', `
((kernel) => kernel.do_init(kernel, {}))
({${meta_kernel}})`], {
                    cwd: config.cwd || '.',
                    stdio: [process.stdin, process.stdout, process.stderr, 'ipc']
                })
            session.child.on('message', message => {
                // assign the responder
                var result = {};
                var execution_count = Object.values(message)[0].execution_count;
                delete Object.values(message)[0].execution_count;
                result[Object.keys(message)[0]] = {
                    execution_count,
                    content: Object.values(message)[0],
                    respond: responders[execution_count]
                };
                return session.message(kernel, session, result);
            });
            return setupSockets(config).then(sockets => {
                session.sockets = sockets;
                console.log('connecting sockets');
                sockets.heartbeat.on('message', sockets.heartbeat.send);
                
                sockets.control.on('message', parseMessage.bind(null,
                    session.message.bind(session, kernel, session),
                    collapseMessage.bind(null, config.key, session.respond.bind(session, kernel, session))));
                sockets.shell.on('message', parseMessage.bind(null,
                    session.message.bind(session, kernel, session),
                    collapseMessage.bind(null, config.key, session.respond.bind(session, kernel, session))));
                // TODO: finish this
                sockets.stdin.on('message', parseMessage.bind(null,
                    session.input_request.bind(session, kernel, session),
                    collapseMessage.bind(null, config.key, session.respond.bind(session, kernel, session))));
                // iopub appears to be write to only
                if(typeof session.startup === 'function') session.startup(kernel, session);
            })
        },
        startup: (kernel, session) => console.log('kernel started'),
        message: (kernel, session, message) => {
            console.log(JSON.stringify(message));
            if(message.clear) message.respond({clear_output: message.clear})
            if(message.input) message.respond({input_request: message.input})
            if(typeof session[Object.keys(message)[0]] === 'undefined') {
                console.error(`unhandled message type ${JSON.stringify(message)}`);
                return;
            }
            session[Object.keys(message)[0]].apply(session, [kernel, session].concat(Object.values(message)));
        },
        respond: (kernel, session, message, encoded) => {
            console.log(`response ${JSON.stringify(message)}`);
            if(Object.keys(message)[0] === 'shutdown_reply')
                session.sockets.control.send(encoded);
            else if(Object.keys(message)[0].substr(-6) === '_reply')
                session.sockets.shell.send(encoded);
            else if(Object.keys(message)[0] === 'input_request')
                session.sockets.stdin.send(encoded);
            else
                session.sockets.iopub.send(encoded);
        },
        status: (kernel, session, request) => console.log(`child status ${JSON.stringify(request.content)}`),
        display: (kernel, session, request) => {
            request.respond(request.display_id
                    ? {update_display_data: {metadata: {}, data: request.content, transient: request.display_id}}
                    : {display_data: {metadata: {}, data: request.content}})
        },
        mime: (kernel, session, request) => {
            request.respond({execute_reply: {
                status: 'ok',
                execution_count: request.execution_count,
                payload: [], // TODO(NR) not implemented,
                user_expressions: {}, // TODO(NR) not implemented,
            }})
            request.respond({execute_result: {
                execution_count: request.execution_count,
                data: request.content,
                metadata: {}
            }})
        },
        comm_info_request: (kernel, session, request) => {
            request.respond({status: {execution_state: 'busy'}});
            request.respond({status: {execution_state: 'idle'}});
            request.respond({comm_info_reply: {comms: {}}});
        },
        input_request: (kernel, session, request) => {
            // TODO: finish this
            //this.onReplies[response.header.msg_id] = onReply;
        },
        shutdown_request: (kernel, session, request) => {
            request.respond({status: {execution_state: 'busy'}});
            return Promise.resolve()
                .then(() => session.child.kill('SIGTERM'))
                .then(() => request.respond({status: {execution_state: 'idle'}}))
                .then(() => request.respond({shutdown_reply: request.content}))
                .then(() => process.exit() /*request.content.restart
                      ? session.init(kernel.config, kernel, session)
                      : void 0*/)
        },
        kernel_info_request: (kernel, session, request) => {
            request.respond({status: {execution_state: 'busy'}});
            request.respond({status: {execution_state: 'idle'}});
            request.respond({kernel_info_reply: kernel.kernel_info});
        },
        is_complete_request: (kernel, session, request) => {
            request.respond({status: {execution_state: 'busy'}});
            var result;
            return Promise.resolve()
                .then(() => kernel.do_is_complete(request.content.code))
                .then(r => result = r)
                .then(() => request.respond({status: {execution_state: 'idle'}}))
                .then(() => request.respond({is_complete_reply: {
                    status: result ? 'complete': 'incomplete',
                    indent: ''
                }}))
        },
        execute_request: (kernel, session, request) => {
            var execution_count = ++count;
            responders[execution_count] = request.respond;
            request.respond({status: {execution_state: 'busy'}});
            request.respond({execute_input: {
                execution_count: execution_count,
                code: request.content.code,
            }})
            var result;
            return Promise.resolve()
                .then(() => kernel.do_execute({
                    code: request.content.code,
                    execution_count
                }))
                .then(r => result = r)
                .then(() => request.respond({status: {execution_state: 'idle'}}))
        },
        complete_request: (kernel, session, request) => {
            request.respond({status: {execution_state: 'busy'}});
            var result;
            return Promise.resolve()
                .then(() => kernel.do_complete(request.content.code, request.content.cursor_pos))
                .then(r => result = r)
                .then(() => request.respond({status: {execution_state: 'idle'}}))
                .then(() => request.respond({complete_reply: {
                    matches: result.completion.list,
                    cursor_start: result.completion.cursorStart,
                    cursor_end: result.completion.cursorEnd,
                    status: "ok",
                }}))
        },
        history_request: (kernel, session, request) => {
            request.respond({status: {execution_state: 'busy'}});
            var result;
            return Promise.resolve()
                .then(() => kernel.do_history(request))
                .then(r => result = r)
                .then(() => request.respond({status: {execution_state: 'idle'}}))
                .then(() => request.respond({history_reply: {
                    history: [] // TODO
                }}))
        },
        inspect_request: (kernel, session, request) => {
            var execution_count = ++count;
            request.respond({status: {execution_state: 'busy'}});
            var result;
            return Promise.resolve()
                .then(() => kernel.do_inspect(request))
                .then(r => result = r)
                .then(() => request.respond({status: {execution_state: 'idle'}}))
                .then(() => request.respond({inspect_reply: {
                    found: true,
                    data: {
                        /*
                        TODO: format inspections
    if (result.inspection) {
        docstring = (
            result.inspection.type + ": " + result.inspection.string
        );
    }

    if (result.doc) {
        docstring = result.doc.usage;
        if (result.doc.usage) {
            docstring += "\n\n" + result.doc.description;
        }
    }*/
                        'text/plain': docstring,
                        'text/html': `<pre>${docstring}</pre>`,
                    },
                    metadata: {},
                    status: "ok",
                }}))
                .catch(e => request.respond({inspect_reply: {
                    status: "error",
                    execution_count: execution_count,
                    ename: result.error.ename,
                    evalue: result.error.evalue,
                    traceback: result.error.traceback,
                }}))
        }
    }, session);
}

module.exports = {
    standardMessaging,
    kernelSessionTemplate
};


reading notebook /Users/briancullinan/jupyter_ops/Core/files.ipynb
compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[0] aka list project files
compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[1] aka common ignore paths
reading notebook /Users/briancullinan/jupyter_ops/Analytics/budgeting.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/documentation.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/reconcile timeline data.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/resume.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Analytics/socializing.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Art/virtual worlds.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/aws.ipynb
reading notebook /Users/briancullinan/jupyte

reading notebook /Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/stack overflow.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/What is Selenium.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Selenium/you earned it.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Untitled.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/bash.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/child process.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/compress.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/dates.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/diff.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Utilities/heartbeat.ipynb
reading notebook /Use

{ standardMessaging: [Function: standardMessaging],
  kernelSessionTemplate: [Function: kernelSessionTemplate] }

get kernel class?


In [ ]:
var importer = require('../Core');
var languageTemplate = importer.import('get kernel language');
var kernelInfoTemplate = importer.import('get kernel info');
var {kernelMetaTemplate} = importer.import('get meta kernel');
var {kernelSessionTemplate} = importer.import('get kernel session');

function kernelTemplate({
    kernel_info,
    meta_kernel,
    language_info,
    session,
    install
}) {
    kernel_info = kernelInfoTemplate(Object.assign(kernel_info, {language_info}));
    meta_kernel = kernelMetaTemplate(meta_kernel);
    var kernel = {
        ...kernel_info,
        kernel_info,
        meta_kernel,
        language_info: languageTemplate(language_info),
        session: kernelSessionTemplate(session),
        install
    };
    var remote_control = Object.keys(meta_kernel)
        .reduce((obj, k) => (obj[k] = ((kernel, message) => {
            var request = {};
            request[k] = message;
            return kernel.session.child.send(request);
        }).bind(kernel, kernel), obj), {});
    return Object.assign(kernel, {...remote_control});
}

module.exports = kernelTemplate;


get javascript kernel?


In [5]:
var importer = require('../Core');
var {installKernel} = importer.import('bind to jupyter zmq sockets');
var kernelTemplate = importer.import('get kernel class');
var {standardMessaging} = importer.import('get kernel session');
var {javascriptMeta} = importer.import('get meta kernel');

var PACKAGE_VERSION = require('../package.json').version;

function getVersion(str) {
    return str.split('.').map(v => parseInt(v, 10)).join('.')
}

/*
var transpile = require("@babel/core").transform.bind({
    presets: [
        [require.resolve("@babel/preset-env"), {
            loose: true,
            targets: {node: true},
        }],
    ],
});
*/

function javascriptKernel(config) {
    var kernel = kernelTemplate({
        kernel_info: {
            protocol_version: '5.1',
            implementation: 'javascript',
            implementation_version: PACKAGE_VERSION,
            banner: 'NodeJS',
            // TODO: automatically create this from installation intructions
            help_links: ['https://nodejs.org']
        },
        language_info: {
            name: 'javascript',
            version: getVersion(process.versions.node),
            file_extension: '.js',
            mimetype: 'application/javascript'
        },
        session: standardMessaging({
            startup: (kernel, session) => {
                return Promise.resolve()
//                    .then(() => kernel.do_execute(`require('@babel/register')`))
//                    .then(() => session.respond({execution_state: 'idle'}))
            }
        }),
        meta_kernel: javascriptMeta({
            /*do_execute: (kernel, request) => {
                var result = new require('vm').Script(transpile(code));
                process.send({mime: result})
            },*/
        }),
        install: installKernel
    })
    if(config === 'install') {
        kernel.install({
            display_name: kernel.banner,
            path: process.argv[2],
            language: kernel.implementation + '_test'
        });
    } else if(typeof config === 'string') {
        config = JSON.parse(fs.readFileSync(config));
        kernel.session.init(kernel, config);
    }
    return kernel;
}

module.exports = javascriptKernel;

if(typeof $$ !== 'undefined') {
    $$.sendResult(javascriptKernel());
}



reading notebook /Users/briancullinan/jupyter_ops/Core/kernels.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Core/kernels.ipynb
compiling /Users/briancullinan/jupyter_ops/Core/kernels.ipynb[4] aka get kernel session
reading notebook /Users/briancullinan/jupyter_ops/Core/kernels.ipynb
compiling /Users/briancullinan/jupyter_ops/Core/kernels.ipynb[1] aka get meta kernel


{ protocol_version: 5.1,
  implementation: 'javascript',
  implementation_version: undefined,
  banner: 'NodeJS',
  help_links: [ 'https://nodejs.org' ],
  language_info:
   { name: 'javascript',
     version: '10.16.0',
     file_extension: '.js',
     mimetype: 'application/javascript' },
  do_init: [Function: do_init],
  do_execute: [Function: do_execute],
  do_shutdown: [Function: do_shutdown],
  do_is_complete: [Function: do_is_complete],
  do_complete: [Function: do_complete],
  do_history: [Function: do_history],
  do_inspect: [Function: do_inspect],
  session:
   { init: [Function: init],
     startup: [Function: startup],
     message: [Function: message],
     respond: [Function: respond],
     display: [Function: display],
     input_request: [Function: input_request],
     shutdown_request: [Function: shutdown_request],
     kernel_info_request: [Function: kernel_info_request],
     is_complete_request: [Function: is_complete_request],
     execute_request: [Function: execu

bind to jupyter zmq sockets?


In [ ]:
var zmq = require("jmp").zmq;
var util = require('util');
var importer = require('../Core');
var {standardJsonTemplate} = importer.import('get kernel json');
var mkdirpSync = importer.import('mkdirp');

function setupSockets(config) {
    const sockets = {
        control: {
            port: config.control_port,
            type: 'xrep',
        },
        shell: {
            port: config.shell_port,
            type: 'xrep',
        },
        stdin: {
            port: config.stdin_port,
            type: 'router',
        },
        iopub: {
            port: config.iopub_port,
            type: 'pub',
        },
        heartbeat: {
            port: config.hb_port,
            type: 'rep',
        }
    }
    var keys = Object.keys(sockets);
    return Promise.all(keys.map(s => setupSocket.apply(null, [sockets[s], config])))
        .then(sockets => sockets.reduce((obj, socket, i) => (obj[keys[i]] = socket, obj), {}))
}

function setupSocket(config, general) {
    const sock = zmq.socket(config.type);
    const addr = general.transport + '://' + general.ip + ':' + config.port
    return util.promisify(sock.bind.bind(sock))(addr)
        .then(() => sock)
}

function installKernel(configJson) {
    console.log(`installing kernel ${JSON.stringify(configJson)}`);
    mkdirpSync(`./.kernel/${configJson.language}`);
    require('fs').writeFileSync(`./.kernel/${configJson.language}/kernel.json`,
                                JSON.stringify(standardJsonTemplate(configJson), null, 4));
    require('child_process').execSync(`
jupyter kernelspec install --user --replace "./.kernel/${configJson.language}"`);
    require('rimraf').sync('./.kernel');
}

module.exports = {
    setupSockets,
    installKernel
};
